# Event Display

For visualization applications, the user can create an instance of `G4JLEventDisplay([settings file])` and give it to the constructor of `G4JLApplication` in the `evtdisplay` attribute.

The constructor accepts a visualization settings file that will overwrite the default settings in the file [Geant4.jl/ext/G4Vis/settings.jl](https://github.com/JuliaHEP/Geant4.jl/blob/master/ext/G4Vis/settings.jl). The format of the settings is Julia `NamedTuple`. Here is an example:
```julia
(
    display = (
        show_axis = false,
    ),
    trajectories = (
        color = :yellow,
    ),
)
```


Let's define a simple detector and particle gun as generator

In [ ]:
using Geant4
using Geant4.SystemOfUnits
using CairoMakie, Rotations, IGLWrap_jll # to force loading G4Vis extension

In [ ]:
struct SimpleDetector <: G4JLDetector ; end

#---Materials----------------------------------------------------------------------------------
nist = G4NistManager!Instance()
m_air = FindOrBuildMaterial(nist, "G4_AIR")
m_bgo = FindOrBuildMaterial(nist, "G4_BGO")

#---Volumes------------------------------------------------------------------------------------
worldS = G4Box("world", 10cm, 10cm, 20cm)
worldLV = G4LogicalVolume(worldS, m_air, "World")
worldPV = G4PVPlacement(nothing, G4ThreeVector(), worldLV, "World", nothing, false, 0, false)

crystalS = G4Box("world", 5cm, 5cm, 10cm)
crystalLV = G4LogicalVolume(crystalS, m_bgo, "Crystal")
crystalPV = G4PVPlacement(nothing, G4ThreeVector(), crystalLV, "Crystal", worldLV, false, 0, false)

#---define getConstructor
Geant4.getConstructor(::SimpleDetector)::Function = (::SimpleDetector) -> worldPV

In [ ]:
particlegun = G4JLGunGenerator(particle = "e-", 
                               energy = 1GeV, 
                               direction = G4ThreeVector(0,0,1), 
                               position = G4ThreeVector(0,0,-10cm));

Now define an instance of `G4JLEventDisplay`. The visualization settings file is in same directory as this notebook. 

In [ ]:
#---Event Display----------------------------------------------------------------------------------
evtdisplay   = G4JLEventDisplay(joinpath(@__DIR__, "VisSettings.jl"))

In [ ]:
#---Create the Application-------------------------------------------------------------------------
app = G4JLApplication(detector     = SimpleDetector(),          # simple detector
                      generator    = particlegun,               # primary generator to instantiate
                      physics_type = FTFP_BERT,                 # what physics list to instantiate
                      evtdisplay   = evtdisplay                 # event display
                     )

configure(app)
initialize(app)

In [ ]:
#---Event Display is anow ready display events-----------------------------------------------------
beamOn(app, 1)
display("image/png", evtdisplay.figure)

In [ ]:
beamOn(app, 1)
display("image/png", evtdisplay.figure)